# 📸 Image Labeling GUI (Kanji/Katakana/Hiragana/Other)

This notebook allows you to label images interactively inside Jupyter using buttons.

In [ ]:

import os
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image
import io

# --- SETTINGS ---
image_folder = './images'  # Set your images folder
save_csv_path = 'labels.csv'

# --- LOAD IMAGES ---
image_paths = [os.path.join(dp, f) for dp, dn, filenames in os.walk(image_folder) for f in filenames if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
image_paths.sort()
labels = {}
current_index = 0

# --- WIDGETS ---
image_widget = widgets.Image()
label_text = widgets.Label()

kanji_button = widgets.Button(description='Kanji', button_style='info')
katakana_button = widgets.Button(description='Katakana', button_style='success')
hiragana_button = widgets.Button(description='Hiragana', button_style='warning')
other_button = widgets.Button(description='Other', button_style='danger')
next_button = widgets.Button(description='Next')
back_button = widgets.Button(description='Back')
save_button = widgets.Button(description='Save CSV')

button_box = widgets.HBox([kanji_button, katakana_button, hiragana_button, other_button])
nav_box = widgets.HBox([back_button, next_button, save_button])
out = widgets.Output()

# --- FUNCTIONS ---
def load_image(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        img.thumbnail((400, 400))  # Resize for display
        buf = io.BytesIO()
        img.save(buf, format='PNG')
        return buf.getvalue()

def update_display():
    if 0 <= current_index < len(image_paths):
        image_widget.value = load_image(image_paths[current_index])
        rel_path = os.path.relpath(image_paths[current_index], start='.')
        label_text.value = f"{current_index+1}/{len(image_paths)} : {rel_path}"
    else:
        label_text.value = "No more images."

def label_current(label):
    if 0 <= current_index < len(image_paths):
        labels[image_paths[current_index]] = label

def on_kanji_clicked(b):
    global current_index
    label_current('Kanji')
    current_index += 1
    update_display()

def on_katakana_clicked(b):
    global current_index
    label_current('Katakana')
    current_index += 1
    update_display()

def on_hiragana_clicked(b):
    global current_index
    label_current('Hiragana')
    current_index += 1
    update_display()

def on_other_clicked(b):
    global current_index
    label_current('Other')
    current_index += 1
    update_display()

def on_next_clicked(b):
    global current_index
    if current_index < len(image_paths) - 1:
        current_index += 1
    update_display()

def on_back_clicked(b):
    global current_index
    if current_index > 0:
        current_index -= 1
    update_display()

def on_save_clicked(b):
    rel_paths = [os.path.relpath(p, start='.') for p in labels.keys()]
    df = pd.DataFrame({
        'filepath': rel_paths,
        'label': list(labels.values())
    })
    df.to_csv(save_csv_path, index=False)
    with out:
        clear_output()
        print(f"Labels saved to {save_csv_path}")

# --- LINK BUTTONS ---
kanji_button.on_click(on_kanji_clicked)
katakana_button.on_click(on_katakana_clicked)
hiragana_button.on_click(on_hiragana_clicked)
other_button.on_click(on_other_clicked)
next_button.on_click(on_next_clicked)
back_button.on_click(on_back_clicked)
save_button.on_click(on_save_clicked)

# --- DISPLAY ---
display(image_widget)
display(label_text)
display(button_box)
display(nav_box)
display(out)

update_display()
